# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846730013116                   -0.70    4.8
  2   -7.852297476594       -2.25       -1.53    1.0
  3   -7.852646205854       -3.46       -2.52    3.2
  4   -7.852646678705       -6.33       -3.35    2.5
  5   -7.852646686037       -8.13       -4.81    1.0
  6   -7.852646686724       -9.16       -5.27    3.2
  7   -7.852646686730      -11.22       -6.11    1.8
  8   -7.852646686730      -12.66       -7.44    2.2
  9   -7.852646686730      -15.05       -7.54    2.2
 10   -7.852646686730      -15.05       -8.55    2.0
 11   -7.852646686730   +  -14.75       -9.64    2.5
 12   -7.852646686730   +    -Inf       -8.31    1.0
 13   -7.852646686730      -14.57       -8.22    1.8
 14   -7.852646686730   +  -15.05       -7.79    3.0
 15   -7.852646686730   +    -Inf       -8.55    2.2
 16   -7.852646686730      -14.75       -8.88    1.0
 17   -7.852646686730   +  -14.27       -9.51 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846871244231                   -0.70           4.8
  2   -7.852527793827       -2.25       -1.63   0.80    2.0
  3   -7.852637328623       -3.96       -2.70   0.80    1.0
  4   -7.852646414032       -5.04       -3.29   0.80    2.2
  5   -7.852646672857       -6.59       -4.09   0.80    1.2
  6   -7.852646686389       -7.87       -4.70   0.80    1.8
  7   -7.852646686724       -9.48       -5.52   0.80    1.5
  8   -7.852646686729      -11.28       -6.79   0.80    1.8
  9   -7.852646686730      -12.40       -7.40   0.80    2.8
 10   -7.852646686730      -15.05       -8.28   0.80    1.5
 11   -7.852646686730   +    -Inf       -8.84   0.80    2.0
 12   -7.852646686730   +    -Inf       -9.95   0.80    1.8
 13   -7.852646686730      -14.57      -10.61   0.80    2.2
 14   -7.852646686730   +  -14.75      -11.20   0.80    1.0
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.364083e+01     3.391886e+00
 * time: 0.5088930130004883
     1     1.218643e+00     1.726350e+00
 * time: 0.7754168510437012
     2    -1.727633e+00     1.763779e+00
 * time: 0.8053689002990723
     3    -3.657240e+00     1.589369e+00
 * time: 0.8493309020996094
     4    -5.040456e+00     1.499949e+00
 * time: 0.8933939933776855
     5    -6.524960e+00     1.037728e+00
 * time: 0.9359309673309326
     6    -7.151184e+00     3.838461e-01
 * time: 0.977916955947876
     7    -7.375578e+00     2.112753e-01
 * time: 1.0074548721313477
     8    -7.497848e+00     1.317448e-01
 * time: 1.0383689403533936
     9    -7.609609e+00     1.372735e-01
 * time: 1.0683188438415527
    10    -7.674418e+00     1.025169e-01
 * time: 1.1008808612823486
    11    -7.737962e+00     1.085641e-01
 * time: 1.1307430267333984
    12    -7.776292e+00     1.338638e-01
 * time: 1.1607389450073242
    13    -7.819154e+00     1.072218e-01
 * time: 1.19165205955

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846865260643                   -0.70    4.8
  2   -7.852319033040       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686708                   -2.55
  2   -7.852646686730      -10.66       -5.97
  3   -7.852646686730      -14.57      -12.59


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.4794578518618143e-13
|ρ_newton - ρ_scfv| = 1.3181875466942052e-13
|ρ_newton - ρ_dm|   = 1.9399112120854075e-9
